In [30]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit_aer import AerSimulator

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive inside a session. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# with Session(backend=service.backend("ibmq_qasm_simulator")):
#     result = Sampler().run(circuits).result()

In [31]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, Aer

In [32]:
# Define the initial classical chess board
classical_board = [
    ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'],
    ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
    ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
    ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
]

In [33]:
# Define the quantum chess board using qubits
quantum_board = QuantumRegister(64)
classical_register = ClassicalRegister(64)
qc = QuantumCircuit(quantum_board, classical_register)


In [34]:
# Function to apply a move on the quantum chess board
def apply_move(move):
    from_pos, to_pos = move
    
    # Convert the positions to qubit indices
    from_qubit = from_pos[0] * 8 + from_pos[1]
    to_qubit = to_pos[0] * 8 + to_pos[1]
    
    # Apply the move as a quantum gate
    qc.cx(quantum_board[from_qubit], quantum_board[to_qubit])
    qc.x(quantum_board[from_qubit])
    qc.measure(quantum_board[to_qubit], classical_register[to_qubit])

In [35]:
# Main game loop
while True:
    # Print the classical chess board
    for row in classical_board:
        print(' '.join(row))
    print()
    
    # Get user input for the move or end the game
    move_input = input("Enter the position of the piece you want to move (e.g., 'e2'), or enter 'quit' to end the game: ")
    
    # Check if the player wants to end the game
    if move_input.lower() == "quit":
        print("Game ended.")
        break
    
    # Parse the move input
    if len(move_input) != 5:
        print("Invalid move input. Please provide two positions.")
        continue
    
    from_pos = move_input[:2]
    to_pos = move_input[3:]
    
    from_row, from_col = 8 - int(from_pos[1]), ord(from_pos[0]) - ord('a')
    to_row, to_col = 8 - int(to_pos[1]), ord(to_pos[0]) - ord('a')
    
    # Apply the move on the quantum chess board
    apply_move(((from_row, from_col), (to_row, to_col)))
    
    # Simulate the quantum circuit
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend, shots=1)
    result = job.result()
    counts = result.get_counts(qc)
    
    # Update the classical chess board based on the measurement result
    for i in range(len(quantum_board)):
        if counts.get(i, 0) == 1:
            index = quantum_board.index(i)
            row = index // 8
            col = index % 8
            piece = classical_board[row][col]
            classical_board[row][col] = ' ' if piece.islower() else piece.lower()
            break
    
    # Update the classical chess board with the new positions of the pieces
    classical_board[from_row][from_col], classical_board[to_row][to_col] = ' ', classical_board[from_row][from_col]
    


r n b q k b n r
p p p p p p p p
               
               
               
               
P P P P P P P P
R N B Q K B N R



Enter the position of the piece you want to move (e.g., 'e2'), or enter 'quit' to end the game:  a2 a4


r n b q k b n r
p p p p p p p p
               
               
P              
               
  P P P P P P P
R N B Q K B N R



Enter the position of the piece you want to move (e.g., 'e2'), or enter 'quit' to end the game:  g7 g5


r n b q k b n r
p p p p p p   p
               
            p  
P              
               
  P P P P P P P
R N B Q K B N R



...

r n b q k   n r
p p p p p p   p
          b    
            p  
P              
    B          
  P P P P P P P
R N   Q K B N R



Enter the position of the piece you want to move (e.g., 'e2'), or enter 'quit' to end the game:  g1 g3


r n b q k   n r
p p p p p p   p
          b    
            p  
P              
    B       N  
  P P P P P P P
R N   Q K B   R



Enter the position of the piece you want to move (e.g., 'e2'), or enter 'quit' to end the game:  quit


Game ended.


In [36]:
# Print the updated classical chess board
for row in classical_board:
    print(' '.join(row))
print()

r n b q k   n r
p p p p p p   p
          b    
            p  
P              
    B       N  
  P P P P P P P
R N   Q K B   R

